In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import joblib
from datetime import datetime
import os
import re
import datetime

In [13]:
import pandas as pd
import re

# Load the necessary files
schedule_path = '24_25_sched_raw.csv'  # Replace with the correct path
teams_path = 'teams.csv'  # Replace with the correct path

# Load the schedule and teams data
schedule_df = pd.read_csv(schedule_path)
abbreviations_df = pd.read_csv(teams_path)

# Create the mapping dictionary from teams.csv
mapping_dict = dict(zip(abbreviations_df['Long'].str.strip(), abbreviations_df['Abbrv'].str.strip()))

# Map team names to abbreviations using the correct column names
schedule_df['AwayTeam'] = schedule_df['AWAY'].str.strip().map(mapping_dict)
schedule_df['HomeTeam'] = schedule_df['HOME'].str.strip().map(mapping_dict)

# Check for unmapped values
unmapped_away = schedule_df[schedule_df['AwayTeam'].isna()]
unmapped_home = schedule_df[schedule_df['HomeTeam'].isna()]

# Create the "Game" column
schedule_df['Game'] = schedule_df['DATE'] + ' - ' + schedule_df['AwayTeam'] + ', ' + schedule_df['HomeTeam']

# Apply regex pattern (optional, only if needed to clean existing data)
pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+), ([\w\s]+)'
regex_extract = schedule_df['Game'].str.extract(pattern)
regex_extract.columns = ['Date', 'AwayTeam', 'HomeTeam']

# Fill missing values if regex didn't match
regex_extract['Game'] = regex_extract['Date'] + ' - ' + regex_extract['AwayTeam'] + ', ' + regex_extract['HomeTeam']
schedule_df['Game'] = regex_extract['Game'].fillna(schedule_df['Game'])

# Drop unnecessary columns
columns_to_drop = [col for col in ['Unnamed: 4'] if col in schedule_df.columns]
schedule_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Save the cleaned schedule to a new file
output_path = '24_25_sched.csv'  # Replace with the desired output path
schedule_df.to_csv(output_path, index=False)

# Display the cleaned DataFrame for verification
print("Reformatted Schedule:")
print(schedule_df.head())

# If there are unmapped rows, display them for debugging
if not unmapped_away.empty:
    print("Unmapped Away Teams:")
    print(unmapped_away[['DATE', 'AWAY', 'HOME']])

if not unmapped_home.empty:
    print("Unmapped Home Teams:")
    print(unmapped_home[['DATE', 'AWAY', 'HOME']])

Reformatted Schedule:
    DAY        DATE   EASTERN    LOCAL        AWAY        HOME AwayTeam  \
0  Fri.  2024-10-04   1:00 PM  7:00 PM  New Jersey     Buffalo      NaN   
1  Sat.  2024-10-05  10:00 AM  4:00 PM     Buffalo  New Jersey      NaN   
2  Tue.  2024-10-08   4:30 PM  1:30 PM   St. Louis     Seattle      NaN   
3  Tue.  2024-10-08   7:00 PM  7:00 PM      Boston     Florida      NaN   
4  Tue.  2024-10-08  10:00 PM  8:00 PM     Chicago        Utah      NaN   

  HomeTeam  Game  
0      NaN   NaN  
1      NaN   NaN  
2      NaN   NaN  
3      NaN   NaN  
4      NaN   NaN  
Unmapped Away Teams:
            DATE            AWAY          HOME
0     2024-10-04      New Jersey       Buffalo
1     2024-10-05         Buffalo    New Jersey
2     2024-10-08       St. Louis       Seattle
3     2024-10-08          Boston       Florida
4     2024-10-08         Chicago          Utah
...          ...             ...           ...
1307  2025-04-17         Detroit       Toronto
1308  2025-04-17